In [1]:
import jax.numpy as jnp
from jax import random, grad, vmap
from functools import partial
import matplotlib.pyplot as plt
import jaxgp.regression as gpr
from jaxgp.kernels import RBF, Linear, Constant
import timeit
import time

# Full GPR

## Benchmark memory of fitmatrix and fitvector

## Benchmarking with a constant function

In [2]:
data_sizes = [100, 200, 500, 1000, 2000, 3000, 4000, 5000]

kernel_1 = RBF()
kernel_2 = Linear()

### Timing of the training

In [3]:
# for elem in data_sizes:
#     data_split = (elem, )

#     X_data = jnp.linspace(0.0, 1.0, data_split[0]).reshape(-1,1)
#     Y_data =jnp.ones(data_split[0]).reshape(-1)

#     def test_RBF():
#         model = gpr.ExactGPR(kernel=kernel_1, data_split=data_split, noise=0.1)
#         model.train_regression()
#         model(X_data=X_data, Y_data=Y_data)

#     print(f"N={elem}: {timeit.repeat(test_RBF, repeat=5, number=1)}")

### Timing of eval with constant X_grid for different training sizes

In [4]:
# X = jnp.linspace(0.2, 0.8, 5000).reshape(-1,1)

# for elem in data_sizes:
#     data_split = (elem, )

#     X_data = jnp.linspace(0.0, 1.0, data_split[0]).reshape(-1,1)
#     Y_data =jnp.ones(data_split[0]).reshape(-1)

#     model = gpr.ExactGPR(kernel=kernel_1, data_split=data_split, noise=0.1)
#     model.train_regression()
#     model(X_data=X_data, Y_data=Y_data)
#     model.eval()

#     def test_RBF():
#         model(X)


#     print(f"N={elem}: {timeit.repeat(test_RBF, repeat=5, number=1)}")

# Sparse GPR

### Timing of the training

In [5]:
# for elem in data_sizes:
#     data_split = (elem, )

#     X_data = jnp.linspace(0.0, 1.0, data_split[0]).reshape(-1,1)
#     Y_data =jnp.ones(data_split[0]).reshape(-1,1)
#     X_ref = jnp.linspace(0.0,1.0,data_split[0]//20).reshape(-1,1)

#     def test_RBF():
#         model = gpr.SparseGPR(kernel=kernel_1, data_split=data_split, X_ref=X_ref, noise=0.1)
#         model.train(X_data=X_data, Y_data=Y_data)

#     print(f"N={elem}: {timeit.repeat(test_RBF, repeat=1, number=1)}")

# Try a weird function

In [6]:
key = random.PRNGKey(int(time.time()))

def f(x, noise=0):
    return jnp.sin(2*x**2 + 1/(x+0.5/jnp.pi))*x + noise*random.normal(key,x.shape)

gradient = vmap(grad(f), in_axes=(0,None))

def df(x, noise=0):
    return gradient(x, noise) + noise*random.normal(key,x.shape)

# def f(x, noise=0.0):
#     temp = jnp.where(x<0, -x, x)
#     return temp + noise*random.normal(key,x.shape)

# def df(x, noise=0.0):
#     temp = jnp.where(x<0, -1.0, 1.0)
#     return temp + noise*random.normal(key,x.shape)

In [7]:
bounds = jnp.array([0.0, jnp.pi])
# bounds = jnp.array([-1.0, 1.0])

noise = 0.1

length_scale = 1.0

num_datapoints = 2
datagrid = jnp.linspace(*bounds,num_datapoints)
data_values = f(datagrid,noise)
num_derivpoints = (100,)
# derivative_grid = jnp.linspace(*bounds,num_derivpoints[0])
derivative_grid = random.uniform(key, num_derivpoints, minval=bounds[0], maxval=bounds[1])
derivative_values = df(derivative_grid,noise)
data_split = (num_datapoints, ) + num_derivpoints

X_data = jnp.concatenate((datagrid,derivative_grid)).reshape(-1,1)
Y_data = jnp.concatenate((data_values,derivative_values)).reshape(-1)

num_referencepoints = 30
# referencegrid = jnp.linspace(*bounds,num_referencepoints).reshape(-1,1)
referencegrid = random.uniform(key, (num_referencepoints,), minval=bounds[0], maxval=bounds[1]).reshape(-1,1)

num_gridpoints = 101
grid = jnp.linspace(*bounds,num_gridpoints)

kernel = Constant() * RBF()# + Linear()
kernel_params = (length_scale, length_scale)#, 1.0, 1.0)

In [8]:
# model = gpr.ExactGPR(kernel,data_split,kernel_params,noise=noise)
# model.train(X_data,Y_data)
# (means_full, stds_full) = model.eval(grid.reshape(-1,1),True)
# means_full = means_full.reshape(-1)
# stds_full = stds_full.reshape(-1)

# model = gpr.SparseGPR(kernel,data_split,referencegrid,kernel_params,noise=noise)
# model.train(X_data,Y_data)
# (means_ppa, stds_ppa) = model.eval(grid.reshape(-1,1),True)
# means_ppa = means_ppa.reshape(-1)
# stds_ppa = stds_ppa.reshape(-1)

model = gpr.ExactGPR(kernel,data_split,kernel_params,noise=noise)
model.train_regression()
model(X_data,Y_data)
model.eval()
means_full, stds_full = model(grid.reshape(-1,1))
means_full = means_full.reshape(-1)
stds_full = stds_full.reshape(-1)

model = gpr.SparseGPR(kernel,data_split,referencegrid,kernel_params,noise=noise)
model.train_regression()
model(X_data,Y_data)
model.eval()
(means_ppa, stds_ppa) = model(grid.reshape(-1,1))
means_ppa = means_ppa.reshape(-1)
stds_ppa = stds_ppa.reshape(-1)

NameError: name 'test' is not defined

In [ ]:
print(stds_full)

[0.0108251  0.01042112 0.01166267 0.01178974 0.01149796 0.01131507
 0.01130453 0.01138334 0.01140427 0.01143558 0.01141472 0.01139904
 0.01142516 0.01141472 0.01138334 0.01134138 0.01127813 0.01126227
 0.01125168 0.01124638 0.01126227 0.01127285 0.01127813 0.01124638
 0.01119858 0.01123047 0.01124638 0.01124108 0.01123047 0.01125168
 0.01125168 0.01124108 0.01122516 0.01125168 0.01123047 0.01124638
 0.01123577 0.01121985 0.01120922 0.0112039  0.01120922 0.01122516
 0.01121985 0.01121454 0.01121454 0.0112039  0.01122516 0.01121454
 0.01121985 0.01119325 0.01119858 0.01117727 0.0112039  0.01119858
 0.01119858 0.01119325 0.0112039  0.01117727 0.01121454 0.01117727
 0.01117727 0.01116126 0.01114523 0.01114523 0.01119325 0.01122516
 0.01119858 0.01115592 0.01110236 0.01105933 0.01106472 0.01106472
 0.01109162 0.01108086 0.01107548 0.01107548 0.01106472 0.01105933
 0.0110701  0.0110701  0.0110701  0.01104315 0.0110701  0.01106472
 0.01105933 0.01105933 0.01106472 0.01106472 0.01106472 0.0110

In [ ]:
plt.plot(grid,means_full,label="full")
plt.fill_between(grid,means_full-2*stds_full,means_full+2*stds_full,alpha=0.5)
plt.plot(grid,means_ppa,label="ppa")
plt.fill_between(grid,means_ppa-2*stds_ppa,means_ppa+2*stds_ppa,alpha=0.5)
plt.plot(grid,f(grid,0.0),color="grey",ls="--",label="true function")
plt.scatter(datagrid,data_values,color="r",label="function values")
plt.scatter(derivative_grid,jnp.zeros_like(derivative_grid),label="deriv points")
plt.scatter(referencegrid.reshape(-1), -jnp.ones_like(referencegrid.reshape(-1)), label="refpoints")
plt.grid()
plt.legend()